In [3]:
import pandas as pd
import os
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
import datasets as dt
import numpy as np
import tqdm
import torch


'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
Tapaco_dataset=load_dataset("csv",data_files="tapaco_paraphrases_dataset_2.csv")
Tapaco_dataset
dataset_train=Tapaco_dataset["train"]
dataset_train.shuffle()
dataset=dataset_train.train_test_split(test_size=10)
dataset    

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 67.20it/s]


Dataset csv downloaded and prepared to C:/Users/Lenovo/.cache/huggingface/datasets/csv/default-acc1a73b7981a46d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 95.00it/s]


DatasetDict({
    train: Dataset({
        features: ['Text', 'Paraphrase'],
        num_rows: 90
    })
    test: Dataset({
        features: ['Text', 'Paraphrase'],
        num_rows: 10
    })
})

In [5]:
dataset["train"]["Paraphrase"]

['My brother watches television.',
 'What are you so nervous about?',
 'By the way, Mike, please tell me how to get to your house.',
 'What are you so worried about?',
 'Spring is in the air.',
 'What made you angry?',
 "That's my money.",
 'I eat some cheese.',
 'Spring will be here soon.',
 "I'm not a madman.",
 "I don't even have time for reading.",
 'I have no time even for reading.',
 "I'm not stupid.",
 'Do you know how to speak English?',
 'Spring has arrived early.',
 'Are you speaking English?',
 'When the father came home, I watched TV.',
 'She refuses to say more about it.',
 'Can you not speak English?',
 'Why is he sad?',
 "When the cat's away, the mice do play.",
 'Where is the librarian?',
 'What made you so angry?',
 'Spring has come.',
 'Why are you guys so angry?',
 'You don’t need to apply in advance.',
 "I'm not a moron.",
 'Why are you upset?',
 "I'm not crazy.",
 'I owe her money.',
 'Can you speak English?',
 "I'm eating cheese.",
 'That which is easily acquired 

In [8]:
model_checkpoint="t5-base"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)
max_input_length=512 #512 ve 1024 arasında değişiyor maksimum izin verilen uzunluk , garanti olsun diye 512 yaptım.

def tokenizer_function(example) :
 
    model_inputs = tokenizer(example["Text"], max_length=max_input_length, truncation=True)

    outputs = tokenizer(example["Paraphrase"], max_length=max_input_length, truncation=True)

    model_inputs["labels"] = outputs["input_ids"]
    return model_inputs

tokenized_dataset=dataset.map(tokenizer_function, batched=True)


Loading cached processed dataset at C:\Users\Lenovo\.cache\huggingface\datasets\csv\default-acc1a73b7981a46d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0395256018f6c96c.arrow


In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Paraphrase', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 90
    })
    test: Dataset({
        features: ['Text', 'Paraphrase', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})

In [5]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
batch_size = 5
model_name = "t5_Paraphrase_model_TapacoDataset"
model_dir = f"./paraphrase_model/{model_name}"

args =  TFTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    
) 

In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer)
metric = dt.load_metric("rouge")


In [8]:
def metrics_display(eval_pred):
    predictions, labels = eval_pred
    decoded_predictions= tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
                      
    result = metric.compute(predictions=decoded_predictions, references=decoded_labels,
                            use_stemmer=True)
    return {k: v for k, v in result.items()}
   

In [ ]:
trainer = Seq2SeqTrainer(
    model_init=AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device),
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer.to(device),
    compute_metrics=metrics_display,
)

In [ ]:
trainer.train()

In [ ]:
model_name=
model_dir="./paraphrase_model/"
trainer.save_model(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
max_input_length = 512

import locale
print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding #locale kodunu hazır buldum encoding hatası almıştım utf-8'e sabitlemek için kullandım



In [ ]:
def paraphrase_generator(sentence):
  
    input_sent = [sentence]
    inputs = tokenizer(input_sent, max_length=max_input_length, truncation=True, return_tensors="pt")
    outputs = model.generate(**inputs, num_beams=5, do_sample=True, max_length=512)
    final_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    paraphrase = decoded_output[0]
    print("paraphrase of this sentence is: " , paraphrase)